#그룹 전체 취향 반영하기

In [8]:
import json
import psycopg2
import pandas as pd
import numpy as np

In [9]:
def load_db():
  with open("/content/drive/MyDrive/yeoreodigm/data_files/db_info.json") as json_file:
    DB_INFO = json.load(json_file)
  
  endpoint = DB_INFO["ENDPOINT"]
  dbname = DB_INFO["DB_NAME"]
  user = DB_INFO["USER_ID"]
  password = DB_INFO["PASSWORD"]
  db = psycopg2.connect(host=endpoint,dbname=dbname,user=user,password=password)
  return db

In [17]:
db = load_db()

In [ ]:
entire_taste = np.zeros(shape=(1,22))
entire_taste

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0.]])

In [14]:
import datetime

def retrieve_taste_db(user_id,db):
  cursor = db.cursor()
  sql = f"SELECT taste,last_update from user_taste where user_id={user_id}"
  cursor.execute(sql)
  result = cursor.fetchone()
  return result

def calculate_taste(user_id,db):
  print("calculate_tast...")
  cursor = db.cursor()
  #1. retrieve from survey
  load_survey_sql = f'SELECT result FROM survey_result WHERE member_id = {user_id};'
  cursor.execute(load_survey_sql)
  result = cursor.fetchone()
  survey_result = result[0]
  print(survey_result)

  theme_score = np.array([0]*22)
  theme_score = theme_score.reshape(1,-1)
  for place in survey_result:
    load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
    cursor.execute(load_place_sql)
    place_theme = cursor.fetchone()
    
    theme_score += np.array(place_theme)*10
  #2. retrieve from place-like log
  load_like_log_sql = f'SELECT place_id FROM place_like WHERE member_id = {user_id};'
  cursor.execute(load_like_log_sql)
  user_like_log = cursor.fetchall()
  if user_like_log:
    user_like_log = list(map(lambda x: x[0],user_like_log))
    for place in user_like_log:
      load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
      cursor.execute(load_place_sql)
      place_theme = cursor.fetchone()
      theme_score += np.array(place_theme)*7
  
  #3. retrieve from place_click log
  load_click_log_sql = f'SELECT place_id FROM places_log WHERE member_id = {user_id};'
  cursor.execute(load_click_log_sql)
  user_click_log = cursor.fetchall()
  if user_click_log:
    user_click_log = list(map(lambda x: x[0],user_click_log))
    for place in user_click_log:
      load_place_sql = f'SELECT theme FROM places_analysis WHERE place_id={place}'
      cursor.execute(load_place_sql)
      place_theme = cursor.fetchone()
      theme_score += np.array(place_theme)
  print("calculate_tast DONE!!")
  print("user's taste : ",theme_score.reshape(1,-1))
  return theme_score.reshape(1,-1)

def retrieve_taste(user_id,db):
  info = retrieve_taste_db(user_id,db)
  cursor =db.cursor()
  if info != None: 
    if (datetime.date.today() - info[1].date()).days>=3:
      print("update calcuate")
      updated_taste = calculate_taste(user_id,db)
      print("updated_taste: ",updated_taste)
      db_updated_taste = list(map(int, updated_taste[0]))
      print("db_updated_taste : ",db_updated_taste)
      #sql = "update user_taste set taste = %s where user_id = %s,"(db_updated_taste,user_id)
      #update db
      cursor.execute("update user_taste set taste = %s, last_update = %s where user_id = %s",(db_updated_taste,datetime.datetime.now() , user_id))
      db.commit()
    else:
      print("db에서 그대로 가져와서 빨라용!!")
      return np.array(info[0]).reshape(1,-1)
  else:
    print("calculate new")
    updated_taste = calculate_taste(user_id,db)
    db_updated_taste = list(map(int, updated_taste[0]))
    #insert new row to db
    cursor.execute("INSERT INTO user_taste(user_id,taste) VALUES(%s, %s)",(user_id,db_updated_taste))
    db.commit()
    
  return updated_taste

def retrieve_group_taste(user_list,db):
  entire_taste = np.zeros(shape=(1,22))
  for user in user_list:
    now_taste = retrieve_taste(user_id=user,db=db)
    print('now taste : ',now_taste[0])
    entire_taste = entire_taste + now_taste
  print("entire_taste before dividing: ", entire_taste)  
  entire_taste = entire_taste / len(user_list)

  return entire_taste



##n명 취향 평균내기

In [ ]:
user_list = [184,2845,2155]
check_result = retrieve_group_taste(user_list,db)

db에서 그대로 가져와서 빨라용!!
now taste :  [107  56 188 191 108  17  60 357 369 106  67  79 330 133 213 192  73  64
 411  67   8  56]
db에서 그대로 가져와서 빨라용!!
db에서 그대로 가져와서 빨라용!!
now taste :  [ 2  2  8  8  0  2  2 16  6  8  6  4 12  4 10  2  2  6 16  4  0  0]
db에서 그대로 가져와서 빨라용!!
db에서 그대로 가져와서 빨라용!!
now taste :  [ 22  30 112 101  15  20  49 181 156  73  36  39 154  72 133  56  42  34
 189  53   8  15]
db에서 그대로 가져와서 빨라용!!
entire_taste before dividing:  [[131.  88. 308. 300. 123.  39. 111. 554. 531. 187. 109. 122. 496. 209.
  356. 250. 117. 104. 616. 124.  16.  71.]]


In [ ]:
check_result

array([[ 43.66666667,  29.33333333, 102.66666667, 100.        ,
         41.        ,  13.        ,  37.        , 184.66666667,
        177.        ,  62.33333333,  36.33333333,  40.66666667,
        165.33333333,  69.66666667, 118.66666667,  83.33333333,
         39.        ,  34.66666667, 205.33333333,  41.33333333,
          5.33333333,  23.66666667]])

##user가 1명만 오는 경우

In [5]:
sample_user = list(map(int,"184".split(',')))

In [15]:
db = load_db()
check_only_one = retrieve_group_taste(sample_user,db)

db에서 그대로 가져와서 빨라용!!
now taste :  [107  56 188 191 108  17  60 357 369 106  67  79 330 133 213 192  73  64
 411  67   8  56]
entire_taste before dividing:  [[107.  56. 188. 191. 108.  17.  60. 357. 369. 106.  67.  79. 330. 133.
  213. 192.  73.  64. 411.  67.   8.  56.]]


In [16]:
check_only_one

array([[107.,  56., 188., 191., 108.,  17.,  60., 357., 369., 106.,  67.,
         79., 330., 133., 213., 192.,  73.,  64., 411.,  67.,   8.,  56.]])